# 1. Buzzer control

In [ ]:
import os
os.system("sudo chmod 777 /dev/ttyTHS1")

In [ ]:
import RPi.GPIO as GPIO

BEEP_pin = 6 # buzzer pin number

GPIO.setmode(GPIO.BCM)
GPIO.setup(BEEP_pin, GPIO.OUT, initial = GPIO.LOW) # starting option

In [ ]:
import time

def Beep(beeptime):
    GPIO.output(BEEP_pin, GPIO.HIGH) # buzzer on
    time.sleep(beeptime) # sound time
    GPIO.output(BEEP_pin, GPIO.LOW) # buzzer off

In [ ]:
Beep(0.3)

 # 2. Automatic avoid

In [ ]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))

In [ ]:
device = torch.device('cuda')
model = model.to(device)

In [ ]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224,fps=10)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blocked_slider]))

In [ ]:
from jetbot import Robot

robot = Robot()

In [ ]:
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    
    if prob_blocked > 0.7 :
        robot.stop()
        Beep(1)

        robot.left(2)
        time.sleep(0.5)
        robot.stop()
        robot.forward(2)
        time.sleep(0.5)
        robot.stop()
        robot.right(2)
        time.sleep(0.5)
        robot.stop()
    else :
        robot.stop()
        robot.forward(0.2)
        time.sleep(0.5)
        robot.stop()
        return 1
        
update({'new': camera.value})  # we call the function once to intialize

In [ ]:
while(1) :  # robot acting
    if update({'new': camera.value}) == 0:
    print('robot blocked.')
    else :
        print('automatic avoided.')
        break